In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

#from sklearn.preprocessing import LabelEncoder


#  loading the data set 
file = pd.read_csv('BSOM_DataSet_for_HW2.csv')

df=file[['all_mcqs_avg_n20','all_NBME_avg_n4','LEVEL']]

df['LEVEL'].fillna(value='A', inplace=True) 

#df.fillna(df['STEP_1'].mean(), inplace=True)
LEVEL = {'A': 0,'B': 1,'C':2,'D':3}

df.LEVEL = [LEVEL[item] for item in df.LEVEL] 
#print(df) 




In [2]:
X= np.array(df.iloc[:,0:2])
#X_reg = (X - X.mean())/(X.max()-X.min())
#X = np.c_[np.ones((len(X), 1)), X]

target=df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,target, test_size=0.30, random_state =0)


print(X_test)

[[0.837  0.8525]
 [0.857  0.8725]
 [0.7    0.6825]
 [0.798  0.8475]
 [0.749  0.7925]
 [0.809  0.8525]
 [0.778  0.8375]
 [0.666  0.7475]
 [0.759  0.81  ]
 [0.746  0.7625]
 [0.755  0.77  ]
 [0.807  0.8125]
 [0.807  0.88  ]
 [0.841  0.81  ]
 [0.762  0.8075]
 [0.799  0.8125]
 [0.868  0.895 ]
 [0.817  0.86  ]
 [0.728  0.8075]
 [0.649  0.67  ]
 [0.769  0.795 ]
 [0.764  0.79  ]
 [0.9    0.9125]
 [0.793  0.86  ]
 [0.741  0.7175]
 [0.795  0.8225]
 [0.637  0.6825]
 [0.706  0.72  ]
 [0.682  0.74  ]
 [0.806  0.82  ]
 [0.744  0.745 ]
 [0.91   0.905 ]
 [0.886  0.925 ]
 [0.788  0.8   ]
 [0.679  0.715 ]]


In [3]:

def sigmoid(x):
    return 1/(1 + np.exp(-x))
def Mapping(i,y_train):
    ymapped=np.where(y_train == i, 1, 0)
    return ymapped
def  Cost(X_train,theta,y_mapp,lambdaa):
    y_pred =  sigmoid(X_train.dot(theta))
    cost = np.sum((-y_mapp * np.log(y_pred)) - ((1-y_mapp)*np.log(1-y_pred)))/ len(X_train) 
    reg_cost= cost + (lambdaa/(2*len(X_train))) * sum(np.square(theta))
    return reg_cost

def gradient_Descent(X_train,y_train,epochss,alphaa,lambdaa):
    
    X_train = np.c_[np.ones((len(X_train), 1)), X_train]
    theta = []
    cost_value=[]
    thetas_inside =  np.random.rand(3,1) 
    # here i in range 4 since we have four class labels
    for i in range(4):
        
        y_mapp=Mapping(i,y_train)
        cost_each = np.zeros(epochss)
        
        e= 0
            
        while (e < epochss):

            cost_each[i] = Cost(X_train,thetas_inside,y_mapp,lambdaa)
            ynot = sigmoid(np.dot(X_train,(thetas_inside)))
            c1=np.sum(np.dot(X_train.T,(ynot - y_mapp))+(lambdaa * thetas_inside))
            thetas_inside = thetas_inside -(alphaa/ len(X_train)) *(c1)
            thetas_inside[0] = thetas_inside[0]- (alphaa / len(X_train)) * np.dot(X_train.T,(ynot - y_mapp))[0]
            e+=1
        theta.append(thetas_inside)
        cost_value.append(cost_each)


    return theta



def LogisticRegression( X_train, y_train,epochss,alphaa,lambdaa):

    y_train = y_train.to_numpy().reshape(len(X_train),1)
    thetas_pred=gradient_Descent(X_train,y_train,epochss,alphaa,lambdaa)
    return thetas_pred
      
        
thetas1=LogisticRegression(X_train,y_train,10000,0.1,0.001)
thetas2=LogisticRegression(X_train,y_train,10000,0.1,3.0)
thetas3=LogisticRegression(X_train,y_train,10000,0.1,6.0)
thetas4=LogisticRegression(X_train,y_train,10000,0.1,7.8)
thetas5=LogisticRegression(X_train,y_train,10000,0.1,9.9)


In [4]:
def Predict(X_test,thetas1):
    
    X = np.c_[np.ones((len(X_test), 1)), X_test]
    thetaA=[]
    thetaB=[]
    thetaC=[]
    thetaD=[]

  


    val1_sig=np.matrix(thetas1[0].T)*np.array(X.T)
    val1=sigmoid(val1_sig)
    val2_sig=np.matrix(thetas1[1].T)*np.array(X.T)
    val2=sigmoid(val2_sig)

    val3_sig=np.matrix(thetas1[2].T)*np.array(X.T)
    val3=sigmoid(val3_sig)
    val4_sig=np.matrix(thetas1[3].T)*np.array(X.T)
    val4=sigmoid(val4_sig)
    val2_df=pd.DataFrame(val2)
    val3_df=pd.DataFrame(val3)
    val4_df=pd.DataFrame(val4)
    belongs_to=pd.DataFrame(val1)


    belongs_to=pd.concat([belongs_to,val2_df], ignore_index=True)
    belongs_to=pd.concat([belongs_to,val3_df], ignore_index=True)
    belongs_to=pd.concat([belongs_to,val4_df], ignore_index=True)

    belongs_to_T=belongs_to.T

    

    values=belongs_to_T.idxmax(axis=1)
    return values

values1=Predict(X_test,thetas1)
values2=Predict(X_test,thetas2)
values3=Predict(X_test,thetas3)
values4=Predict(X_test,thetas4)
values5=Predict(X_test,thetas5)


In [5]:
print('The accuracy score for lambda = 0.001, alpha =10000,learning rate=0.1 ',accuracy_score(y_test, values1))
print('The accuracy score for lambda = 3, alpha =10000,learning rate=0.1',accuracy_score(y_test, values2))
print('The accuracy score for lambda = 6, alpha =10000,learning rate=0.1',accuracy_score(y_test, values3))
print('The accuracy score for lambda = 7.8, alpha =10000,learning rate=0.1',accuracy_score(y_test, values4))
print('The accuracy score for lambda = 9.9, alpha =10000,learning rate=0.1',accuracy_score(y_test, values5))


The accuracy score for lambda = 0.001, alpha =10000,learning rate=0.1  0.45714285714285713
The accuracy score for lambda = 3, alpha =10000,learning rate=0.1 0.42857142857142855
The accuracy score for lambda = 6, alpha =10000,learning rate=0.1 0.4
The accuracy score for lambda = 7.8, alpha =10000,learning rate=0.1 0.4
The accuracy score for lambda = 9.9, alpha =10000,learning rate=0.1 0.4


In [6]:
print('For lambda = 0.001, alpha =10000,learning rate=0.1')
print('confusion_matrix')
print(confusion_matrix(y_test, values1))
print('accuracy_score',accuracy_score(y_test, values1))
print(classification_report(y_test, values1))
print('f1_score',f1_score(y_test, values1, average="macro"))
print('precision_score',precision_score(y_test, values1, average="macro"))
print('recall_score',recall_score(y_test, values1, average="macro"))  

For lambda = 0.001, alpha =10000,learning rate=0.1
confusion_matrix
[[ 0  7  0  0]
 [ 0 14  0  0]
 [ 0 10  2  0]
 [ 0  2  0  0]]
accuracy_score 0.45714285714285713
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.42      1.00      0.60        14
           2       1.00      0.17      0.29        12
           3       0.00      0.00      0.00         2

   micro avg       0.46      0.46      0.46        35
   macro avg       0.36      0.29      0.22        35
weighted avg       0.51      0.46      0.34        35

f1_score 0.22036474164133737
precision_score 0.3560606060606061
recall_score 0.2916666666666667


In [7]:
print('For lambda = 3, alpha =10000,learning rate=0.1')
print('confusion_matrix')
print(confusion_matrix(y_test, values2))
print('accuracy_score',accuracy_score(y_test, values2))
print(classification_report(y_test, values2))
print('f1_score',f1_score(y_test, values2, average="macro"))
print('precision_score',precision_score(y_test, values2, average="macro"))
print('recall_score',recall_score(y_test, values2, average="macro"))  

For lambda = 3, alpha =10000,learning rate=0.1
confusion_matrix
[[ 2  5  0  0]
 [ 1 13  0  0]
 [ 0 12  0  0]
 [ 0  2  0  0]]
accuracy_score 0.42857142857142855
              precision    recall  f1-score   support

           0       0.67      0.29      0.40         7
           1       0.41      0.93      0.57        14
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         2

   micro avg       0.43      0.43      0.43        35
   macro avg       0.27      0.30      0.24        35
weighted avg       0.30      0.43      0.31        35

f1_score 0.24130434782608695
precision_score 0.26822916666666663
recall_score 0.3035714285714286


In [8]:
print('For lambda = 6.0, alpha =10000,learning rate=0.1')
print('confusion_matrix')
print(confusion_matrix(y_test, values3))
print('accuracy_score',accuracy_score(y_test, values3))
print(classification_report(y_test, values3))
print('f1_score',f1_score(y_test, values3, average="macro"))
print('precision_score',precision_score(y_test, values3, average="macro"))
print('recall_score',recall_score(y_test, values3, average="macro"))  

For lambda = 6.0, alpha =10000,learning rate=0.1
confusion_matrix
[[ 0  7  0  0]
 [ 0 14  0  0]
 [ 0 12  0  0]
 [ 0  2  0  0]]
accuracy_score 0.4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.40      1.00      0.57        14
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         2

   micro avg       0.40      0.40      0.40        35
   macro avg       0.10      0.25      0.14        35
weighted avg       0.16      0.40      0.23        35

f1_score 0.14285714285714288
precision_score 0.1
recall_score 0.25


In [9]:
print('For lambda = 7.8, alpha =10000,learning rate=0.1')
print('confusion_matrix')
print(confusion_matrix(y_test, values4))
print('accuracy_score',accuracy_score(y_test, values4))
print(classification_report(y_test, values4))
print('f1_score',f1_score(y_test, values4, average="macro"))
print('precision_score',precision_score(y_test, values4, average="macro"))
print('recall_score',recall_score(y_test, values4, average="macro"))  

For lambda = 7.8, alpha =10000,learning rate=0.1
confusion_matrix
[[ 0  7  0  0]
 [ 0 14  0  0]
 [ 0 12  0  0]
 [ 0  2  0  0]]
accuracy_score 0.4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.40      1.00      0.57        14
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         2

   micro avg       0.40      0.40      0.40        35
   macro avg       0.10      0.25      0.14        35
weighted avg       0.16      0.40      0.23        35

f1_score 0.14285714285714288
precision_score 0.1
recall_score 0.25


In [10]:
print('For lambda = 9.9, alpha =10000,learning rate=0.1')
print('confusion_matrix')
print(confusion_matrix(y_test, values5))
print('accuracy_score',accuracy_score(y_test, values5))
print(classification_report(y_test, values5))
print('f1_score',f1_score(y_test, values5, average="macro"))
print('precision_score',precision_score(y_test, values5, average="macro"))
print('recall_score',recall_score(y_test, values5, average="macro"))  

For lambda = 9.9, alpha =10000,learning rate=0.1
confusion_matrix
[[ 0  7  0  0]
 [ 0 14  0  0]
 [ 0 12  0  0]
 [ 0  2  0  0]]
accuracy_score 0.4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.40      1.00      0.57        14
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         2

   micro avg       0.40      0.40      0.40        35
   macro avg       0.10      0.25      0.14        35
weighted avg       0.16      0.40      0.23        35

f1_score 0.14285714285714288
precision_score 0.1
recall_score 0.25
